***이미 수행된 전처리***

1. 한문 제거 또는 대체

  한문 -> 현대 한국어로 정제

2. 결측치, 이상치 제거

  제목, 내용 모두 결측치 제거(4151->4078)

3. 일부 시 내용 보완 및 추가

  같은 제목이라도 일부 내용 수정 → 재작성 및 보완

전처리3 - '내용' 영문,일문 버전 제거 및 '제목' 시인정보 제거

In [25]:
import pandas as pd
import re

In [26]:
df = pd.read_csv('poems_pp_ver2.csv', encoding='utf-8-sig')

In [27]:
# 제목 + 내용이 모두 중복된 시 제거 (첫 번째만 남김)
duplicated = df.drop_duplicates(subset=['제목', '내용'], keep='first').copy()

In [28]:
# 불필요한 앞뒤 공백 및 줄바꿈 정리 (\n, \t)
duplicated['제목'] = duplicated['제목'].str.strip()
duplicated['내용'] = duplicated['내용'].str.strip().str.replace('\n+', '\n', regex=True)

In [29]:
duplicated.to_csv("poems_before_tagging.csv", index=False, encoding='utf-8-sig')

전처리4 - 감정/주제 메타태깅

In [ ]:
pip install transformers torch

KcELECTRA: ELECTRA 기반 프리트레인 언어모델

- ELECTRA

  BERT와 비슷한 구조의 Transformer 기반 모델

  "마스킹 후 복원" 대신 "진짜/가짜 구분" 학습 방식 사용 => 더 효율적이고 학습 속도 빠름, 소규모에서도 좋은 성능

- beomi/KcELECTRA-base는 네이버 뉴스 댓글, 영화 리뷰, 위키피디아 등의 비속어/감성 데이터에 특화된 사전학습 모델

  감정 분류, 욕설 탐지, 혐오 발언 필터링 등 **텍스트 분류(task-specific fine-tuning)**에 주로 사용

In [ ]:
# 감정 분석 메타태깅
from transformers import pipeline
import pandas as pd

df = pd.read_csv("poems_before_tagging.csv", encoding='utf-8-sig')

# 감정 분석 파이프라인 생성 (기존 KoELECTRA 모델)
sentiment_pipeline = pipeline("sentiment-analysis", model="beomi/KcELECTRA-base")

# 감정 분석 함수 정의
def get_emotion(text):
    try:
        # 250자까지 분석 (모델 입력 제한 고려, 일반적으로 200~400, max: 500)
        result = sentiment_pipeline(text[:250])[0]
        return result['label']
    except:
        return "Unknown"

label_map = {
    'LABEL_0': '부정',
    'LABEL_1': '긍정',
    'LABEL_2': '중립'
}

# 데이터프레임 적용 (라벨 변환 매핑)
df['감정_raw'] = df['내용'].apply(get_emotion)
df['감정'] = df['감정_raw'].map(label_map).fillna(df['감정_raw'])

df.to_csv("poems_with_emotion.csv", index=False)

In [ ]:
# 주제(키워드 기반) 분석 메타태깅
# 키워드 사전 정의
theme_dict = {
    '자연': ['하늘', '바람', '꽃', '구름', '산', '강'],
    '사랑': ['사랑', '연인', '눈빛', '그리움'],
    '죽음': ['죽음', '이별', '장례', '눈물'],
}

# 주제 추출 함수
def extract_theme(text):
    for theme, keywords in theme_dict.items():
        if any(kw in text for kw in keywords):
            return theme
    return '기타'

df['주제'] = df['내용'].apply(extract_theme)